# 1. Fuzzy matching for Mexican new IDs with multiple old IDs and multimodal distribution in websites for each new ID

In [1]:
# Import relevant libraries
import pandas as pd 
from thefuzz import fuzz
from thefuzz import process
import numpy as np
import json
import unidecode
from itertools import combinations

## 1.1 Read Builtwith Websites and Panjiva Names Data  - Multimodal Distribution
Read data of new IDs with multiple old IDs that have a multimodal distribution of websites for each new ID

In [2]:
# Read data of builtwith website
mult_builtwith_ind = pd.read_csv("../../Data/Mexico/processed_data/new_ids_cases_for_fuzzy_builtwith_MEX.csv")
mult_builtwith_ind.head(30)

,old_id,new_id,panjiva_raw_firm_name,builtwith_website,different_old_ids,n_url,n_total_old_ids,mode_url_count,share_url_mode,obs_has_mode,is_multimodal,unimodal,only_one_url_retrieved
0,MEX167255,MEX11792,PROMAQUIN SA DE CV,promaquin.com,True,1,2,1,NaN,1,1,0,0
1,MEX11792,MEX11792,PROMAQUINASA DE CV,promsacorp.com,True,1,2,1,NaN,1,1,0,0
2,MEX123788,MEX123788,"DIICSA INFRAESTRUCTURA, SA DE CV",diicsacv.com,True,1,2,1,NaN,1,1,0,0
3,MEX97820,MEX123788,"DICA INFRAESTRUCTURA,S.A. DE C.V.",grupodica.com,True,1,2,1,NaN,1,1,0,0
4,MEX125129,MEX125129,AMERICA LOGISTICS SA DE CV,americalogistics.com,True,1,2,1,NaN,1,1,0,0
5,MEX99792,MEX125129,"AMERICA LOGISTIC GROUP, S.A. DE C.V.",americalogisticsgroup.com,True,1,2,1,NaN,1,1,0,0
6,MEX13316,MEX13316,MERCK SHARP & DOHME COMERCIALIZADORAS DE RL DE CV,bravecto.com,True,1,2,1,NaN,1,1,0,0
7,MEX4792,MEX13316,MERCK SHARP & DOHME DE MEXICO SA DE CV,msd.com,True,1,2,1,NaN,1,1,0,0
8,MEX14343,MEX14343,"ROCHE INDUSTRIES PROFESSIONAL DE MEXICO, S. DE...",mexcentrix.com,True,1,2,1,NaN,1,1,0,0
9,MEX117707,MEX14343,"ROCHE DC MEXICO, S.A. DE C.V.",roche.com,True,1,2,1,NaN,1,1,0,0


## 1.2. Preprocessing

For the preprocessing step, we use the following procedure:

1. **Normalization and Cleaning**:
   - **Lowercase Conversion and Accent Removal**: All text data, including company names and website URLs, are converted to lowercase to standardize the format and ensure uniformity in processing. Additionally, accents are removed using `unidecode`. These accents are often found in company names but rarely in website URLs.
   - **Removal of Common Business Suffixes**: Business-related suffixes such as "SA DE CV", "S.A. DE C.V.", and others are removed from company names. This step is crucial for aligning company names more closely with their website, which typically does not include such formal identifiers.

2. **Character Filtration**:
   - **Non-Alphanumeric Character Removal**: All non-alphanumeric characters, including spaces, commas, periods, and hyphens, are stripped from both names and URLs. This reduces noise and prevents mismatches that can occur due to irrelevant characters or spacing issues in the data.

3. **Extraction and Utilization of Initials**:

    - **Initials Extraction**: This step involves selecting the first letter of each significant word in the company's name to form a set of initials. Words considered insignificant, such as common conjunctions, prepositions, or those shorter than three letters, are excluded ('la', 'el', 'de', 'los', etc.). For instance:
      - For `MERCK SHARP & DOHME COMERCIALIZADORAS DE RL DE CV`, the significant words are `MERCK`, `SHARP`, `DOHME`, and `COMERCIALIZADORAS`.
      - The initials extracted from these words are `MSDC`, which represents a concise representation of the company's name.
    - **Usage in Comparisons**: These initials, "MSDC", can be directly compared to website names. In this case, the website "msd.com" shares the initials "MSD", which is a partial match to the extracted initials from the company name. This commonality in initials suggests a strong linkage between the company name and the website, enhancing the accuracy of matching in scenarios where such abbreviations or acronyms are used in domain names. 

    This method of using initials helps in effectively matching company names with their corresponding websites, especially when websites utilize acronyms or initials as their domain names.

4. **Website URL Simplification**:
   - **Domain Simplification**: Common web prefixes (like "www.") and suffixes (such as ".com", ".org", ".net") are removed from URLs to focus on the core part of the domain, which is essential for direct comparisons with company names.
   - **Conditional Geographic Indicator Removal**: The presence of geographic indicators like ".mx" or ".mex" is conditionally managed based on the content of the company name. If terms related to "Mexico" (such as "mexico" or "mex") are found in the cleaned company name, these indicators are preserved in the URL to enhance specificity in matching. Conversely, if such indicators are absent, they are removed to prevent potential false associations and simplify the domain further.

5. **Application of Conditional Logic**:
   - **Geographic Relevance**: The logic implemented ensures that geographic relevance is maintained by preserving country-specific domain extensions like ".mx" or ".mex" when the company name suggests a Mexican association. This approach helps in accurately identifying and matching companies with their respective web domains, especially in localized contexts.

By implementing these preprocessing steps, the goal is to standardize and optimize the input data for more effective matching, especially when using fuzzy logic or other matching algorithms. This preparation is key to minimizing discrepancies and enhancing the accuracy of identifying matches between company names and their corresponding websites.

In [3]:
def extract_initials(name):
    # Extract initials from significant words
    words = [word for word in name.split() if word not in {'de', 'la', 'las', 'los', 'y', 'e', 'el', 'del', 'of', 'and', ".", ",", "&"}]
    return ''.join([word[0] for word in words])

def preprocess(name, website):
    # Normalize: lowercase and remove accents
    name = unidecode.unidecode(name.lower())
    website = unidecode.unidecode(website.lower())
    
    # Remove common suffixes and other non-alphanumeric characters from name and website
    suffixes = ["sa de cv", "s a de c v", "s de rl de cv", "sa cv", "s de rl", " sa ", "s. a. de c. v.",  
                "s.a.", "s.a. de c.v.", "inc", "corp", "llc", "r.l", "c.v", "s. de r.l.", " de ", " s.a. ", " s. ", 
               " de c.v.", "s.a. de c.v.", "sa de c v"]
    for suffix in suffixes:
        if suffix in [" de "]:
            name = name.replace(suffix, ' ')
        else:
            name = name.replace(suffix, '') # Do not leave space

    
    # Initials are extracted from the cleaned name
    initials = extract_initials(name)
    name = ''.join(e for e in name if e.isalnum())  # Remove all non-alphanumeric characters from name
    
    # Determine if 'mexico' or similar words are in the cleaned company name to decide how to clean the website URL
    if any(x in name for x in ['mexico', 'mex']):
        website = website.replace('.mx', '.mex')
        website = website.replace('www.', '').replace('.com', '').replace('.org', '').replace('.net', '').replace('.gob', '')
    else:
        website = website.replace('www.', '').replace('.mx', '').replace('.mex', '').replace('.com', '').replace('.org', '').replace('.net', '').replace('.gob', '')

    # Remove all remaining non-alphanumeric characters from website
    website = ''.join(e for e in website if e.isalnum())
    return name, website, initials


## 1.3. Scoring algorithm for string similarity between Builtwith websites and Panjiva names when there is a multimodal distribution of websites by new ID


#### Levenshtein Distance and FuzzyWuzzy

The `fuzzywuzzy` Python package, which is utilized in this code, leverages the Levenshtein distance to calculate differences between sequences (i.e., text strings). This method measures the minimum number of single-character edits (insertions, deletions, or substitutions) required to change one word (string) into another. The package provides various ways to compare strings, which are essential in determining the similarity and thereby assisting in matching company names to their respective websites.


#### Normalization of Similarity Scores

Similarity scores are normalized percentages derived from the Levenshtein distance. They are calculated as follows:

$$\text{Similarity Score} = 100 \times \left(1 - \frac{\text{Levenshtein distance}}{\text{maximum string length}} \right)$$

A similarity score of 100 indicates an exact match between strings, whereas a score closer to 0 indicates greater dissimilarity.

#### Key Scores Used
1. **Ratio**: This score calculates the standard Levenshtein distance similarity between two strings on a scale from 0 to 100. It’s useful for direct, full-string comparisons.
2. **Partial Ratio**: Compares partial segments of the string, focusing on the best matching substring. This is particularly useful when matching initials or abbreviations to longer strings.
3. **Token Sort Ratio**: Compares two strings by sorting their tokens alphabetically and then joining them back into a single string, thus removing the impact of word order.
4. **Token Set Ratio**: Compares strings based on shared words, regardless of order or additional words. It splits strings into tokens, finds common words, combines them with the best matching unique tokens, then calculates similarity. This method is ideal for matching variations of names that have the same core words but may include extras or be in a different order.


#### Process for Identifying the Best Website Match

1. **Preprocessing**:
   - Convert each company name and website URL to lowercase.
   - Remove accents from characters.
   - Strip common business suffixes.
   - Remove non-alphanumeric characters.

2. **Scoring**:
   - Compare the preprocessed website against the preprocessed company name and the initials of the preprocessed company name.
   - Compute four distinct similarity scores (ratio, partial ratio, token sort ratio, token set ratio) between the cleaned company name and the cleaned website.
   - Calculate the **Average score** by averaging the above four scores.
   - Calculate the **Initials score** by comparing the cleaned website with the initials of the preprocessed company name using only the partial ratio score.
   - Decide which score to use (Average score or Initials score) based on the following conditions:
     * The Initials score must be greater than 85.
     * The Initials score must exceed the Average score.
     * The string length of the initials must be more than 1 character.
     * The cleaned website's length must be less than 6 characters.
     
    If any condition is not met, the Average score is used; otherwise, the Initials score is considered.

3. **Best Website Determination**:
   - The website with the highest applicable score (Initials score if all conditions are met, otherwise Average score) is identified as the best match for the given old id.


#### Practical Examples

1. **Example 1: Prioritizing Initials for Matching**

    - Raw Company Name: "MERCK SHARP & DOHME COMERCIALIZADORAS DE RL DE CV"
    - Cleaned Company Name: "mercksharpdohmecomercializadora"
    - Extracted Initials: "msdc"
    - Potential Websites: ["msd.com", "bravecto.com"]
    - Cleaned Websites: ["msd", "bravecto"]

    In this example, the comparison process assesses the similarity between the company initials and the potential website domains. For "msd.com," the `partial_ratio` score between the initials "msdc" and the domain "msd" is likely to be higher than the `ratio` score between the cleaned company name and the domain. This is because the initials "msdc" directly correlate with the domain "msd," reflecting a common practice where a company's web domain is an abbreviation of its full name.

    The comparison between the cleaned full company name "mercksharpdohmecomercializadora" and the domain "msd" may result in a lower similarity score (average score) due to the length and complexity of the full name, which could introduce more potential points of difference as measured by the Levenshtein distance.

    Therefore, in cases where a high `partial_ratio` score is achieved with initials, it suggests a deliberate choice of website domain to represent the company's abbreviated name, making the initials to domain comparison the stronger indicator for a match. This approach, which favors initials when they yield a high score, is especially useful when the domain name is likely to be an acronym or initialism of the company name.

- **Example 2: Full Name Match Example**:

    - Raw Name: "FLUID CONTROL MEXICO S DE RL DE CV"
    - Cleaned Name: "fluidcontrolmexico"
    - Initials: "fcm"
    - Websites: "fluidcontrolmx.com" and "pfccorp.com"
    - Cleaned Websites: "fluidcontrolmx" and "pfccorp"
    
    In this example, the initials "fcm" do not closely match the cleaned websites "fluidcontrolmx" or "pfccorp", resulting in a lower partial_ratio score in both cases. Conversely, a comparison between the cleaned name "fluidcontrolmexico" and the cleaned website "fluidcontrolmx" reveals a higher degree of similarity due to the extended overlap of characters. Thus, in this scenario, the full name comparison with the cleaned website is more likely to yield a higher score than the initials comparison, leading to a preference for the full name match with "fluidcontrolmx.com" over "pfccorp.com".

In [4]:
# Define thresholds 
th_init = 80        # Threshold to define when to use initials over the avg_score
th_max_score = 50   # Threshold to define if the max score of the best website by new_id-old is greater than or not to this threshold

In [5]:
# Empty dictionary to store results 
scores_builtwith = {}
# Array with new IDs
new_ids = mult_builtwith_ind["new_id"].unique()

# Iterate over new ids
for new_id in new_ids:
    # Filter dataset to get only observations of old ids associated with that new id 
    df_filtered = mult_builtwith_ind[mult_builtwith_ind["new_id"] == new_id]
    # Create an empty dictionary for the new id
    scores_builtwith[new_id] = {}
    # Create an empty list to store websites with the highest scores
    best_websites = []
    # Iterate over old ids-raw names associated with the new id
    for old_id, raw_name in zip(df_filtered.old_id.to_numpy(), df_filtered.panjiva_raw_firm_name.to_numpy()):
        # Create an empty dictionary for each old id 
        scores_builtwith[new_id][old_id] = {}
        # Assign raw name to the old id dictionary 
        scores_builtwith[new_id][old_id][raw_name] = {}  
        
        max_score = 0
        best_website = None
        dict_websites = {}

        
        # Iterate over websites 
        for website in df_filtered.builtwith_website.to_numpy():
            
            # Preprocess raw name and website and get initials of the cleaned name 
            clean_name, clean_website, initials = preprocess(raw_name, website)
            
            # Does the clean website contain the word "mexico" and the cleaned name does not? If yes, add "mexico" to clean name. 
            # This will give a high score to websites containing the mexico part. 
            if "mexico" in clean_website and "mexico" not in clean_name:
                clean_name = clean_name + "mexico"

            # Add cleaned website to dictionary of websites where the key is the non-cleaned website
            dict_websites[website] = clean_website
            
            
            # Scores comparing cleaned names and cleaned websites 
            score_ratio = fuzz.ratio(clean_name, clean_website)
            score_partial_ratio = fuzz.partial_ratio(clean_name, clean_website)
            score_token_sort_ratio = fuzz.token_sort_ratio(clean_name, clean_website)
            score_token_set_ratio = fuzz.token_set_ratio(clean_name, clean_website)
            
            # Average score 
            avg_score = (0.30*score_ratio + 0.40*score_partial_ratio + 0.15*score_token_sort_ratio + 0.15*score_token_set_ratio)
            
            # Scores comparing initials vs websites (only use partial_ratio as it specifically
            # helps to focus the match on any substring within the website domain that aligns
            # best with the initials)
            initials_score = fuzz.partial_ratio(initials, clean_website)
            
           # Compute highest score
            if (initials_score >= th_init) and (initials_score > avg_score) and len(clean_website) <= 7 and (len(clean_website) <= (len(initials) + 3)) and initials_score > max_score and len(initials) > 1: 
                max_score = initials_score                                                                 # If the cleaned website is too large compared to the initials length then probably the website is not initials based. The length of the clean website must be smaller or equal than the length of the initials + 3 
                best_website = website 
                initials_score_used = 1
                avg_score_used = 0 
                
            else:
                if avg_score > max_score: 
                    max_score = avg_score
                    best_website = website
                    initials_score_used = 0 
                    avg_score_used = 1
            
 
            scores_builtwith[new_id][old_id][raw_name][website] = {
                "cleaned_name": clean_name,
                "cleaned_website": clean_website,
                "initials_name": initials,
                "score_ratio": score_ratio,
                "score_partial_ratio": score_partial_ratio,
                "score_token_sort_ratio": score_token_sort_ratio,
                "score_token_set_ratio": score_token_set_ratio,
                "avg_score": avg_score, 
                "score_initials_partial_ratio": initials_score, 
                "score_initials_used": initials_score_used, 
                "avg_score_used": avg_score_used

            } 
            
        
        # Check if all cleaned websites are equal, if yes, give priority to the website ending in .in
        if len(set(dict_websites.values())) == 1:
            for w in dict_websites.keys():
                if any(substring in w for substring in [".mex", ".mx"]):
                    best_website = w # Give priority to website with ".in"
                else:
                    continue
        
                
        # Store the best website and its score for the current old_id
        scores_builtwith[new_id][old_id]['best_website'] = {
            "website": best_website,
            "max_score": max_score, 
            "is_max_initials_score": initials_score_used, 
            "is_max_avg_score": avg_score_used

        }
        
        # Add the best website to the list 
        best_websites.append(best_website)
        
    if len(set(best_websites)) == 1:
        scores_builtwith[new_id]["share_best_website"] = 1     
    else:
        scores_builtwith[new_id]["share_best_website"] = 0     

In [6]:
#print(json.dumps(scores_builtwith, indent=4, sort_keys=True))

In [7]:
# Create an empty list to store data for DataFrame
data_list = []

# Iterate through the dictionary to extract required information
for new_id, old_ids in scores_builtwith.items():
    for old_id, contents in old_ids.items():
        if old_id == "share_best_website":
            continue
        for panjiva_raw_name, websites in contents.items():
            if panjiva_raw_name == "best_website":
                continue
            for website, info in websites.items():
                data_list.append({
                    "new_id": new_id,
                    "old_id": old_id,
                    "panjiva_raw_name": panjiva_raw_name,
                    "cleaned_name": info["cleaned_name"],
                    "initials_name": info["initials_name"],
                    "website": website,
                    "cleaned_website": info["cleaned_website"],
                    "avg_score": info["avg_score"],
                    "score_partial_ratio": info["score_partial_ratio"],
                    "score_ratio": info["score_ratio"],
                    "score_token_sort_ratio": info["score_token_sort_ratio"],
                    "score_token_set_ratio": info["score_token_set_ratio"],
                    "score_initials_partial_ratio": info["score_initials_partial_ratio"],
                    "score_initials_used": info["score_initials_used"],
                    "avg_score_used": info["avg_score_used"],
                    "website_with_highest_score": contents["best_website"]["website"],
                    "max_score": contents["best_website"]["max_score"],
                    "old_ids_share_best_website": old_ids["share_best_website"], 
                    "is_max_score_initials_score": contents["best_website"]["is_max_initials_score"], 
                    "is_max_score_avg_score": contents["best_website"]["is_max_avg_score"]

                })

                
# Create a DataFrame from the list of dictionaries
df_builtwith_multimodal = pd.DataFrame(data_list)


# Columns to put at the beginning 
front_columns = ['new_id', 'old_id', 'panjiva_raw_name', 'website', 'website_with_highest_score', 
                 'cleaned_name', 'initials_name',  
                 'cleaned_website',  'old_ids_share_best_website', 'max_score',
                 "is_max_score_avg_score", "is_max_score_initials_score"]

# Remaining columns
remaining_columns = [col for col in df_builtwith_multimodal.columns if col not in front_columns]

# New column order
new_order = front_columns + remaining_columns

# Reorder the DataFrame
df_builtwith_multimodal = df_builtwith_multimodal[new_order]

df_builtwith_multimodal

,new_id,old_id,panjiva_raw_name,website,website_with_highest_score,cleaned_name,initials_name,cleaned_website,old_ids_share_best_website,max_score,is_max_score_avg_score,is_max_score_initials_score,avg_score,score_partial_ratio,score_ratio,score_token_sort_ratio,score_token_set_ratio,score_initials_partial_ratio,score_initials_used,avg_score_used
0,MEX11792,MEX167255,PROMAQUIN SA DE CV,promaquin.com,promaquin.com,promaquin,p,promaquin,1,100.0,1,0,100.0,100,100,100,100,100,0,1
1,MEX11792,MEX167255,PROMAQUIN SA DE CV,promsacorp.com,promaquin.com,promaquin,p,promsacorp,1,100.0,1,0,58.6,67,53,53,53,100,0,1
2,MEX11792,MEX11792,PROMAQUINASA DE CV,promaquin.com,promaquin.com,promaquina,p,promaquin,1,97.0,1,0,97.0,100,95,95,95,100,0,1
3,MEX11792,MEX11792,PROMAQUINASA DE CV,promsacorp.com,promaquin.com,promaquina,p,promsacorp,1,97.0,1,0,56.8,67,50,50,50,100,0,1
4,MEX123788,MEX123788,"DIICSA INFRAESTRUCTURA, SA DE CV",diicsacv.com,diicsacv.com,diicsainfraestructura,di,diicsacv,1,63.2,1,0,63.2,86,48,48,48,100,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,MEX93331,MEX90205,IACNA MEXICO SERVICE COMPANY,iacgroup.com,cna.org.mx,iacnamexicoservicecompany,imsc,iacgroup,1,63.4,1,0,43.6,55,36,36,36,57,0,1
238,MEX94771,MEX62589,CESCO SA,cesco.com,cesco.com,cescosa,cs,cesco,1,89.8,1,0,89.8,100,83,83,83,67,0,1
239,MEX94771,MEX62589,CESCO SA,intercesco.com,cesco.com,cescosa,cs,intercesco,1,89.8,1,0,68.6,83,59,59,59,50,0,1
240,MEX94771,MEX94771,"CESCO IMPORTACIONES,S.A.DE C.V.",cesco.com,cesco.com,cescoimportacionesde,ci,cesco,1,64.0,1,0,64.0,100,40,40,40,67,0,1


In [8]:
# Share of new ids sharing the best website
df_builtwith_multimodal.drop_duplicates("new_id").old_ids_share_best_website.value_counts()/df_builtwith_multimodal.drop_duplicates("new_id").old_ids_share_best_website.value_counts().sum()

1    0.770833
0    0.229167
Name: old_ids_share_best_website, dtype: float64

In [9]:
# Threshold to define if website is correctly assigned
# Are all the max scores above the threshold? 
df_builtwith_multimodal["all_max_scores_above_threshold"] = df_builtwith_multimodal.groupby("new_id")["max_score"].transform(lambda x: (x > th_max_score).all()).astype(int)

# If the max score is above the threshold, then assign website to the new id
df_builtwith_multimodal["assign_website"] = ((df_builtwith_multimodal["all_max_scores_above_threshold"] == 1) & (df_builtwith_multimodal["old_ids_share_best_website"] == 1)).astype(int)

In [10]:
df_builtwith_multimodal.assign_website.value_counts()

1    121
0    121
Name: assign_website, dtype: int64

In [11]:
# Save data 
df_builtwith_multimodal.to_stata( 
    path = "../../Data/Mexico/processed_data/fuzzy_builtwith_websites_scores_MEX.dta",
    variable_labels = {
        "new_id": "New ID",
        "old_id": "Old ID", 
        "panjiva_raw_name": "Panjiva raw name", 
        "cleaned_name": "Cleaned Panjiva raw name", 
        "initials_name": "Initials of cleaned Panjiva name", 
        "website": "Builtwith website", 
        "cleaned_website": "Cleaned Builtwith website", 
        "avg_score": "Avg score(partial, partial ratio, token sort, set ratio)", 
        "score_partial_ratio": "Score from partial ratio", 
        "score_ratio": "Score from ratio", 
        "score_token_sort_ratio": "Score from token sort ratio", 
        "score_token_set_ratio": "Score from token set ratio", 
        "sum_scores": "Scores sum (partial, ratio, token sort and token set ratio)",
        "score_initials_partial_ratio": "Partial ratio score using initials", 
        "score_initials_used": "Is initials-based score used? (1=Yes,0=No)", 
        "avg_score_used": "Is the average score used? (1=Yes,0=No)",
        "website_with_highest_score": "Website with highest score for the given old ID", 
        "max_score": "Score of the website with highest score", 
        "old_ids_share_best_website": "Do old IDs share same top website? (1=Yes,0=No)", 
        "is_max_score_initials_score": "Is the max score a initials-based score? (1=Yes,0=No)",
        "is_max_score_avg_score": "Is the max score the average score? (1=Yes,0=No)", 
        "all_max_scores_above_threshold":"For a given new ID, are all the max scores above the threshold?", 
        "assign_website": "Based on text similarity, is best website correctly assigned?(1=Yes,0=No)"
    }, 
    write_index = False
)


### 1.4 Multimodal cases with no clear best website 

For multimodal cases where there is no clear "best" website, we will create a similarity score between websites (no longer between websites and names). To compare websites we will use Token Set Ratio. This score, in particular, would likely be the most effective as it is designed to handle cases where the strings have a substantial amount of overlapping tokens but might include additional tokens or different ordering. It provides a high degree of flexibility and is more forgiving for minor variations and additional tokens, which is common in URLs.

If the similarity between the websites for a given new ID is too low, we need to manually inspect the case. We might end up dropping them or identifying the correct website to assign to the new ID. Manually checking we can identify two cases: 
- Among the old IDs for the given new ID, there is one old ID that has a correct website, and the other websites are not correct. In such cases, we assign to the new ID the correct website. We will call this case as `non_clear_mode_but_correct_website`. 
- Each old ID has a correct website, i.e., the old IDs with the same new ID are totally different firms, meaning that the error comes from the algorithm that assigns the IDs. For example, Timex Group Precision Engineering Ltd is a manufacturing Engineering company, while Timex Group India is a brand of wrist watches. These cases need to be dropped as they represent different firms. `non_clear_mode_and_non_correct_website`


If the similarity is too high, they are likely different websites of the same firm. In such cases, we would like to define the technology (from the Builtwith dataset) for the given new ID based on all the websites, not just one. For instance, if the e-commerce technology is used on at least one of the websites during a month, but not on the others, then we can simply state that the e-commerce technology for this given new ID is in use, as it was adopted by one of the websites. If it is not used on any website, then we assume that the technology is not in use for that given new ID during that month. We will call this case as `non_clear_mode_but_website_group`.

So based on the above we can define three types of multimodal cases with no clear mode: `non_clear_mode_but_website_group`, `non_clear_mode_but_correct_website`, `non_clear_mode_and_non_correct_website`


For these multimodal cases, we expect to have in the correspondence tables of new IDs the following columns: 


| `new_id` | `panjiva_raw_name` | `correct_builtwith_website` | `similarity_score` |  `type` |  
| -- | -- | -- | --  | --|


Note that for the `non_clear_mode_but_correct_website` case, we will have one observation per new ID. For the `non_clear_mode_and_non_correct_website` we will have multiple rows for a given new id but in any case we will drop these observations from the analysis. For the `non_clear_mode_but_website_group`, we will have in the correspondence table multiple rows for a given new id (different panjiva raw names), but in the analysis we will create the technology group. 

In [12]:
# Dataset of results for multimodal cases wit no clear best website 
no_best_website = df_builtwith_multimodal[df_builtwith_multimodal["old_ids_share_best_website"] == 0]
# List of new IDs with no clear best website 
no_best_website_new_ids = no_best_website.new_id.drop_duplicates().to_list()
no_best_website.head(60)

,new_id,old_id,panjiva_raw_name,website,website_with_highest_score,cleaned_name,initials_name,cleaned_website,old_ids_share_best_website,max_score,...,avg_score,score_partial_ratio,score_ratio,score_token_sort_ratio,score_token_set_ratio,score_initials_partial_ratio,score_initials_used,avg_score_used,all_max_scores_above_threshold,assign_website
8,MEX125129,MEX125129,AMERICA LOGISTICS SA DE CV,americalogistics.com,americalogistics.com,americalogistics,al,americalogistics,0,100.0,...,100.0,100,100,100,100,100,0,1,1,0
9,MEX125129,MEX125129,AMERICA LOGISTICS SA DE CV,americalogisticsgroup.com,americalogistics.com,americalogistics,al,americalogisticsgroup,0,100.0,...,91.6,100,86,86,86,100,0,1,1,0
10,MEX125129,MEX99792,"AMERICA LOGISTIC GROUP, S.A. DE C.V.",americalogistics.com,americalogisticsgroup.com,americalogisticgroup,alg,americalogistics,0,96.8,...,88.6,97,83,83,83,67,0,1,1,0
11,MEX125129,MEX99792,"AMERICA LOGISTIC GROUP, S.A. DE C.V.",americalogisticsgroup.com,americalogisticsgroup.com,americalogisticgroup,alg,americalogisticsgroup,0,96.8,...,96.8,95,98,98,98,67,0,1,1,0
32,MEX201816,MEX149882,MATCO INDUSTRIAL S.A. DE C.V.,matco.com,matco.com,matcoindustrial,mi,matco,0,70.0,...,70.0,100,50,50,50,67,0,1,1,0
33,MEX201816,MEX149882,MATCO INDUSTRIAL S.A. DE C.V.,matcom.com,matco.com,matcoindustrial,mi,matcom,0,70.0,...,65.2,91,48,48,48,67,0,1,1,0
34,MEX201816,MEX201816,MATCOM SA DE CV,matco.com,matcom.com,matcom,m,matco,0,100.0,...,94.6,100,91,91,91,100,0,1,1,0
35,MEX201816,MEX201816,MATCOM SA DE CV,matcom.com,matcom.com,matcom,m,matcom,0,100.0,...,100.0,100,100,100,100,100,0,1,1,0
64,MEX33170,MEX3487,TOYOTA MOTOR MANUFACTURING DE BAJA CALIFORNIA ...,toyota-global.com,toyota.com,toyotamotormanufacturingbajacalifornia,tmmbc,toyotaglobal,0,56.2,...,52.0,70,40,40,40,33,0,1,1,0
65,MEX33170,MEX3487,TOYOTA MOTOR MANUFACTURING DE BAJA CALIFORNIA ...,toyota-tsusho.com,toyota.com,toyotamotormanufacturingbajacalifornia,tmmbc,toyotatsusho,0,56.2,...,50.8,73,36,36,36,33,0,1,1,0


In [13]:
# Create data frame with multimodal cases that we need to analyze 
df_no_best_website_new_ids = mult_builtwith_ind[mult_builtwith_ind["new_id"].isin(no_best_website_new_ids)][["old_id", "new_id", "panjiva_raw_firm_name", "builtwith_website"]]

In [14]:
df_no_best_website_new_ids

,old_id,new_id,panjiva_raw_firm_name,builtwith_website
4,MEX125129,MEX125129,AMERICA LOGISTICS SA DE CV,americalogistics.com
5,MEX99792,MEX125129,"AMERICA LOGISTIC GROUP, S.A. DE C.V.",americalogisticsgroup.com
16,MEX149882,MEX201816,MATCO INDUSTRIAL S.A. DE C.V.,matco.com
17,MEX201816,MEX201816,MATCOM SA DE CV,matcom.com
32,MEX3487,MEX33170,TOYOTA MOTOR MANUFACTURING DE BAJA CALIFORNIA ...,toyota-global.com
33,MEX30910,MEX33170,TOYOTA TSUSHO PROCESSING DE MEXICO SA DE CV,toyota-tsusho.com
34,MEX14562,MEX33170,TOYOTA MOTOR DE MEXICO S. DE R.L. DE C.V.,toyota.com
35,MEX80893,MEX33170,TOYOTA TSUSHO STEEL PIPE DE MEXICO S.A. DE C.V.,toyotafl.com
46,MEX20297,MEX40082,HITACHI DATA SYSTEMS SA DE CV,hds.com
47,MEX66664,MEX40082,HITACHI MEXICO SA DE CV,hitachi-hiemex.com


In [15]:
# Function to clean websites 
def clean_website(website):
    website = website.replace('www.', '').replace('.com', '').replace('.org', '').replace('.net', '').replace('.gob', '').replace('.co', '').replace('.in', '').replace('.gov', '').replace(".mx", '').replace(".mex", "")
    # Remove all remaining non-alphanumeric characters from website
    website = ''.join(e for e in website if e.isalnum())
    
    return website

In [16]:
scores_no_best_website = []
# Iterate over new ids with multimodal cases where there is no a clear best website 
for new_id in no_best_website_new_ids:
    # Filter data frame with new ID to be analyzed 
    df = df_no_best_website_new_ids[df_no_best_website_new_ids["new_id"] == new_id]
    # Identify websites in new id group  
    websites_list = [clean_website(website) for website in df.builtwith_website.to_list()]
    # Create pairs that we need to compared 
    pairs = list(combinations(websites_list, 2))
    score_pairs = []
    # Iterate over pairs
    for pair in pairs:
        s = (fuzz.token_set_ratio(pair[0], pair[1]) + fuzz.partial_ratio(pair[0], pair[1]) + fuzz.token_set_ratio(pair[0], pair[1]) + fuzz.ratio(pair[0], pair[1]))/4
        score_pairs.append(s)
    mean_score = np.mean(score_pairs)
    scores_no_best_website.append(mean_score)

#pd.DataFrame([scores_no_best_website])

In [17]:
# Merge scores with data that only has the multimodal cases where there is no a clear best website
df_no_best_website_new_ids = pd.merge(df_no_best_website_new_ids, 
                                      pd.DataFrame({"new_id":no_best_website_new_ids,
                                                    "scores_no_best_website": scores_no_best_website}),
                                      how="left", on = "new_id")


# Define threshold. Above this threshold we manually check websites 
threshold = 75

# Scores 
df_no_best_website_new_ids["manually_check"] = (df_no_best_website_new_ids["scores_no_best_website"] <= threshold).astype(int)
df_no_best_website_new_ids["create_technology_group"] = (df_no_best_website_new_ids["scores_no_best_website"] > threshold).astype(int)

In [20]:
df_no_best_website_new_ids[df_no_best_website_new_ids["manually_check"] == 1]

,old_id,new_id,panjiva_raw_firm_name,builtwith_website,scores_no_best_website,manually_check,create_website_group
0,MEX123969,MEX123969,LLANTAS Y SERVICIOS DEGA DE SALTILLO SA DE CV,degasaltillo.com,39.500000,1,0
1,MEX119440,MEX123969,LLANTAS Y SERVICIOS DEGA SA DE CV,llantasdega.com,39.500000,1,0
4,MEX132615,MEX132615,"HERRAMIENTAS DAPERMEX, S.A. DE C.V.",dapermex.com,61.000000,1,0
5,MEX60414,MEX132615,"HERRAMIENTAS DIAMEX INDUSTRIA, S.A. DE C.V.",diager.com,61.000000,1,0
6,MEX165655,MEX165655,SISTEMA DE AGUA POTABLE Y ALCANTARILLADO DE LEON,sapal.gob.mx,67.000000,1,0
7,MEX172414,MEX165655,SISTEMA MUNICIPAL DE AGUA POTABLE Y ALCANTARIL...,simapag.gob.mx,67.000000,1,0
8,MEX170761,MEX170761,VIGUETAS Y BOVEDILLAS DE FORMACEL SA DE CV,formacel.com,30.750000,1,0
9,MEX145078,MEX170761,VIGUETAS Y BOVEDILLAS SA DE CV,vibosa.com,30.750000,1,0
12,MEX7326,MEX218958,SUPERMERCADOS INTERNACIONALES H E B SA DE CV,heb.com,0.000000,1,0
13,MEX218958,MEX218958,"SUPERMERCADOS SANTAFE, S.A. DE C.V.",soriana.com,0.000000,1,0


In [18]:
t1 = set(df_no_best_website_new_ids[df_no_best_website_new_ids["manually_check"] == 1].new_id.unique())

In [19]:
# For those cases manually checked, we identify the correct website (if any), if the websites are correct and the error comes 
# from the algorithm that assigns the IDs, then we create a variable called drop_case = 1 
correct_website_mex = pd.DataFrame({
    # New IDs with multimodal cases with no clear best website
    "new_id": ["MEX123969", "MEX132615", "MEX165655", "MEX170761", "MEX218958", "MEX23017", "MEX25271", "MEX31228", "MEX33170", "MEX40082", "MEX58439", "MEX60886", "MEX74960", "MEX7941", "MEX8735", "MEX96799"],  
    # Old id with correct website. If nan it's because none has the correct website or because the firms are totally different firms 
    "old_id_best_website": [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, "MEX3487", "MEX66664", "MEX58439", "MEX60886", "MEX782", "MEX97886", np.nan, np.nan],
    # Manually checked best website. If nan is because the firms are totally different and not just subsidiary of the same company, and therefore the websites are totally different
    "correct_builtwith_website": [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, "toyota-global.com", "hitachi-hiemex.com", "kempro.mx", "fregaderoscm.com", "saint-gobain.com", "bosch.com", np.nan, np.nan],  
    # If websites are totally different because companies with the same new ID are totally different, then create flag to drop those cases 
    "drop_case": [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]
})

In [21]:
t2 = set(correct_website_mex.new_id.unique())

In [22]:
 # Merge the data frame of multimodal cases with no clear best website, with the dataset that has the correct (and manually checked) website (if any)
df_no_best_website_new_ids = pd.merge(df_no_best_website_new_ids, correct_website_mex, on = "new_id", how = "left")

# If missing the drop case is because these are cases that we will not drop and will create a group of technology
df_no_best_website_new_ids["drop_case"] = df_no_best_website_new_ids["drop_case"].fillna(0)

In [23]:
df_no_best_website_new_ids

,old_id,new_id,panjiva_raw_firm_name,builtwith_website,scores_no_best_website,manually_check,create_website_group,old_id_best_website,correct_builtwith_website,drop_case
0,MEX123969,MEX123969,LLANTAS Y SERVICIOS DEGA DE SALTILLO SA DE CV,degasaltillo.com,39.500000,1,0,NaN,NaN,1.0
1,MEX119440,MEX123969,LLANTAS Y SERVICIOS DEGA SA DE CV,llantasdega.com,39.500000,1,0,NaN,NaN,1.0
2,MEX125129,MEX125129,AMERICA LOGISTICS SA DE CV,americalogistics.com,89.500000,0,1,NaN,NaN,0.0
3,MEX99792,MEX125129,"AMERICA LOGISTIC GROUP, S.A. DE C.V.",americalogisticsgroup.com,89.500000,0,1,NaN,NaN,0.0
4,MEX132615,MEX132615,"HERRAMIENTAS DAPERMEX, S.A. DE C.V.",dapermex.com,61.000000,1,0,NaN,NaN,1.0
5,MEX60414,MEX132615,"HERRAMIENTAS DIAMEX INDUSTRIA, S.A. DE C.V.",diager.com,61.000000,1,0,NaN,NaN,1.0
6,MEX165655,MEX165655,SISTEMA DE AGUA POTABLE Y ALCANTARILLADO DE LEON,sapal.gob.mx,67.000000,1,0,NaN,NaN,1.0
7,MEX172414,MEX165655,SISTEMA MUNICIPAL DE AGUA POTABLE Y ALCANTARIL...,simapag.gob.mx,67.000000,1,0,NaN,NaN,1.0
8,MEX170761,MEX170761,VIGUETAS Y BOVEDILLAS DE FORMACEL SA DE CV,formacel.com,30.750000,1,0,NaN,NaN,1.0
9,MEX145078,MEX170761,VIGUETAS Y BOVEDILLAS SA DE CV,vibosa.com,30.750000,1,0,NaN,NaN,1.0


In [24]:
df_no_best_website_new_ids[["old_id", "new_id", "panjiva_raw_firm_name", "builtwith_website", "correct_builtwith_website", "drop_case"]]

,old_id,new_id,panjiva_raw_firm_name,builtwith_website,correct_builtwith_website,drop_case
0,MEX123969,MEX123969,LLANTAS Y SERVICIOS DEGA DE SALTILLO SA DE CV,degasaltillo.com,NaN,1.0
1,MEX119440,MEX123969,LLANTAS Y SERVICIOS DEGA SA DE CV,llantasdega.com,NaN,1.0
2,MEX125129,MEX125129,AMERICA LOGISTICS SA DE CV,americalogistics.com,NaN,0.0
3,MEX99792,MEX125129,"AMERICA LOGISTIC GROUP, S.A. DE C.V.",americalogisticsgroup.com,NaN,0.0
4,MEX132615,MEX132615,"HERRAMIENTAS DAPERMEX, S.A. DE C.V.",dapermex.com,NaN,1.0
5,MEX60414,MEX132615,"HERRAMIENTAS DIAMEX INDUSTRIA, S.A. DE C.V.",diager.com,NaN,1.0
6,MEX165655,MEX165655,SISTEMA DE AGUA POTABLE Y ALCANTARILLADO DE LEON,sapal.gob.mx,NaN,1.0
7,MEX172414,MEX165655,SISTEMA MUNICIPAL DE AGUA POTABLE Y ALCANTARIL...,simapag.gob.mx,NaN,1.0
8,MEX170761,MEX170761,VIGUETAS Y BOVEDILLAS DE FORMACEL SA DE CV,formacel.com,NaN,1.0
9,MEX145078,MEX170761,VIGUETAS Y BOVEDILLAS SA DE CV,vibosa.com,NaN,1.0


In [25]:
ids_ana_to_inspect =  ["MEX123969", "MEX132615", "MEX165655", "MEX170761", "MEX218958", "MEX23017", "MEX25271", "MEX31228", 
                       "MEX33170", "MEX50296",  "MEX8735", "MEX92329", "MEX96799"]


df_ids_ana_to_inspect = df_no_best_website_new_ids[df_no_best_website_new_ids["new_id"].isin(ids_ana_to_inspect)][["new_id", "panjiva_raw_firm_name", "builtwith_website"]]

In [24]:
# Export dataset 
df_no_best_website_new_ids.to_excel("../../Data/Mexico/processed_data/multimodal_cases_no_best_builtwith_website_manually_checked_MEX.xlsx", index = False)

In [26]:
df_ids_ana_to_inspect.to_excel("../../Data/Mexico/processed_data/ids_to_inspect_MEX.xlsx", index = False)

# 2. Fuzzy matching for new IDs with multiple old IDs but with only one URL  retrieved from Builtwith 

In this section, we analyze cases where there are new IDs with multiple old IDs, but it was only possible to retrive the website for one of these old IDs, i.e., only one old ID was matched to Builtwith. 

In [25]:
# Read data of new IDs with multiple old IDs that have only URL retrieved from Builtwith. This is different from the 
# unimodal case, as these are new IDs that have the flag of multiple old IDs -- because in the correspondence table they 
# do have multiple website -- but only the URL of one of these old IDs was able to be retrieved from Builtwith (i.e., only 
# one of these old IDs matched to Builtwith)
only_one_url = pd.read_csv("../../Data/Mexico/processed_data/one_url_retrieved_cases_for_fuzzy_MEX.csv").drop("Unnamed: 0", axis = 1)[["old_id", "new_id", "panjiva_raw_firm_name", "builtwith_website", "only_one_url_retrieved"]]
only_one_url.head(30)

,old_id,new_id,panjiva_raw_firm_name,builtwith_website,only_one_url_retrieved
0,MEX101734,MEX101734,FARMOQUIMIA SA DE CV,adyfarm.mx,1
1,MEX148237,MEX103015,GABLACSA DE CV,gablac.com,1
2,MEX102485,MEX10382,GO GLOBAL S DE RL DE CV,gogloballlc.com,1
3,MEX104130,MEX104130,NEUMATICOS MUEVETIERRA DE PUEBLA SA DE CV,neumaticosmexico.com,1
4,MEX216963,MEX104878,DART INTERNATIONAL S. DE R.L. DE C.V.,dart.biz,1
5,MEX106606,MEX106606,HOTEL ROYAL PLAYACAR SA DE CV,barcelo.com,1
6,MEX10775,MEX10775,DAIDO METAL MEXICO SALES SA DE CV,daidometal.com,1
7,MEX107884,MEX107884,THE AMERICAN SCHOOL FOUNDATION OF GUADALAJARA AC,asfg.mx,1
8,MEX109027,MEX109027,"VST TECHNOLOGY MEXICO, S.A. DE C.V.",vstmexico.com,1
9,MEX102041,MEX1122,CATERPILLAR LOGISTICS SERVICES DE MEXICOS DE R...,caterpillar.com,1


## 2.2. Preprocessing 

We follow the same preprocessing steps as in the section 1.2. to clean Panjiva raw names and Builtwith websites. 

In [26]:
only_one_url[["cleaned_name", "cleaned_website", "initials_name"]] = only_one_url.apply(lambda x: preprocess(x["panjiva_raw_firm_name"], x["builtwith_website"]), axis=1, result_type = "expand")
only_one_url

,old_id,new_id,panjiva_raw_firm_name,builtwith_website,only_one_url_retrieved,cleaned_name,cleaned_website,initials_name
0,MEX101734,MEX101734,FARMOQUIMIA SA DE CV,adyfarm.mx,1,farmoquimia,adyfarm,f
1,MEX148237,MEX103015,GABLACSA DE CV,gablac.com,1,gablac,gablac,g
2,MEX102485,MEX10382,GO GLOBAL S DE RL DE CV,gogloballlc.com,1,goglobal,gogloballlc,gg
3,MEX104130,MEX104130,NEUMATICOS MUEVETIERRA DE PUEBLA SA DE CV,neumaticosmexico.com,1,neumaticosmuevetierrapuebla,neumaticosmexico,nmp
4,MEX216963,MEX104878,DART INTERNATIONAL S. DE R.L. DE C.V.,dart.biz,1,dartinternational,dartbiz,di
...,...,...,...,...,...,...,...,...
217,MEX98127,MEX98127,ACEROMEX GRUPO INDUSTRIAL S.A. DE C.V.,aceromex.com,1,aceromexgrupoindustrial,aceromex,agi
218,MEX55237,MEX99185,HILOS HERMAN S A DE C V,hilosyestambresdemexicosadecv.com,1,hilosherman,hilosyestambresdemexicosadecv,hh
219,MEX114523,MEX99467,GRUPO INNOVADOR SA DE CV,grupoinnovador.com,1,grupoinnovador,grupoinnovador,gi
220,MEX94643,MEX99612,"BUENA VISTA INDUSTRIAL, S.A. DE C.V.",unitoldos.com,1,buenavistaindustrial,unitoldos,bvi


## 2.3 Scoring Algorithm 


In [27]:
# Function to compute Scores
def compute_scores(clean_name, clean_website, initials):
    
    # Scores comparing cleaned names and cleaned websites 
    score_ratio = fuzz.ratio(clean_name, clean_website)
    score_partial_ratio = fuzz.partial_ratio(clean_name, clean_website)
    score_token_sort_ratio = fuzz.token_sort_ratio(clean_name, clean_website)
    score_token_set_ratio = fuzz.token_set_ratio(clean_name, clean_website)
            
    # Average score 
    avg_score = (0.30*score_ratio + 0.40*score_partial_ratio + 0.15*score_token_sort_ratio + 0.15*score_token_set_ratio)
            
    # Scores comparing initials vs websites (only use partial_ratio as it specifically
    # helps to focus the match on any substring within the website domain that aligns
    # best with the initials)
    initials_score = fuzz.partial_ratio(initials, clean_website)
    
    # Max score (define better based on same conditionals)
    if (initials_score > th_init) and (initials_score > avg_score) and len(clean_website) <= 6 and len(initials) > 1: 
        max_score = initials_score
        is_max_score_initials_score = 1
        is_max_score_avg_score = 0

    else: 
        max_score = avg_score
        is_max_score_initials_score = 0
        is_max_score_avg_score = 1

    return score_ratio, score_partial_ratio, score_token_sort_ratio, score_token_set_ratio, avg_score, initials_score, max_score, is_max_score_initials_score, is_max_score_avg_score

In [28]:
# Create variables with scores 
only_one_url[
    ["score_ratio", "score_partial_ratio",
     "score_token_sort_ratio","score_token_set_ratio", 
     "avg_score", "initials_score", "max_score", "is_max_score_initials_score", "is_max_score_avg_score"]
] = only_one_url.apply(lambda x: compute_scores(x["cleaned_name"], x["cleaned_website"], x["initials_name"]), 
                       axis =1, result_type = "expand")


# Threshold to assign website to panjiva raw name:  
only_one_url[["assign_website"]] = (only_one_url[["max_score"]] > th_max_score).astype(int)

In [29]:
only_one_url.assign_website.value_counts()

1    166
0     56
Name: assign_website, dtype: int64

In [30]:
only_one_url

,old_id,new_id,panjiva_raw_firm_name,builtwith_website,only_one_url_retrieved,cleaned_name,cleaned_website,initials_name,score_ratio,score_partial_ratio,score_token_sort_ratio,score_token_set_ratio,avg_score,initials_score,max_score,is_max_score_initials_score,is_max_score_avg_score,assign_website
0,MEX101734,MEX101734,FARMOQUIMIA SA DE CV,adyfarm.mx,1,farmoquimia,adyfarm,f,44.0,73.0,44.0,44.0,55.6,100.0,55.6,0.0,1.0,1
1,MEX148237,MEX103015,GABLACSA DE CV,gablac.com,1,gablac,gablac,g,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,1.0,1
2,MEX102485,MEX10382,GO GLOBAL S DE RL DE CV,gogloballlc.com,1,goglobal,gogloballlc,gg,84.0,100.0,84.0,84.0,90.4,67.0,90.4,0.0,1.0,1
3,MEX104130,MEX104130,NEUMATICOS MUEVETIERRA DE PUEBLA SA DE CV,neumaticosmexico.com,1,neumaticosmuevetierrapuebla,neumaticosmexico,nmp,60.0,83.0,60.0,60.0,69.2,50.0,69.2,0.0,1.0,1
4,MEX216963,MEX104878,DART INTERNATIONAL S. DE R.L. DE C.V.,dart.biz,1,dartinternational,dartbiz,di,42.0,83.0,42.0,42.0,58.4,67.0,58.4,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,MEX98127,MEX98127,ACEROMEX GRUPO INDUSTRIAL S.A. DE C.V.,aceromex.com,1,aceromexgrupoindustrial,aceromex,agi,52.0,100.0,52.0,52.0,71.2,50.0,71.2,0.0,1.0,1
218,MEX55237,MEX99185,HILOS HERMAN S A DE C V,hilosyestambresdemexicosadecv.com,1,hilosherman,hilosyestambresdemexicosadecv,hh,45.0,67.0,45.0,45.0,53.8,67.0,53.8,0.0,1.0,1
219,MEX114523,MEX99467,GRUPO INNOVADOR SA DE CV,grupoinnovador.com,1,grupoinnovador,grupoinnovador,gi,100.0,100.0,100.0,100.0,100.0,67.0,100.0,0.0,1.0,1
220,MEX94643,MEX99612,"BUENA VISTA INDUSTRIAL, S.A. DE C.V.",unitoldos.com,1,buenavistaindustrial,unitoldos,bvi,41.0,47.0,41.0,41.0,43.4,33.0,43.4,0.0,1.0,0


In [31]:
# Save data 
only_one_url.to_stata( 
    path = "../../Data/Mexico/processed_data/fuzzy_aberdeen_names_scores_MEX.dta",
    variable_labels = {
        "new_id": "New ID",
        "old_id": "Old ID", 
        "panjiva_raw_name": "Panjiva raw name", 
        "cleaned_name": "Cleaned Panjiva raw name", 
        "initials_name": "Initials of cleaned Panjiva name", 
        "website": "Builtwith website", 
        "cleaned_website": "Cleaned Builtwith website", 
        "avg_score": "Avg score(partial, partial ratio, token sort, set ratio)", 
        "score_partial_ratio": "Score from partial ratio", 
        "score_ratio": "Score from ratio", 
        "score_token_sort_ratio": "Score from token sort ratio", 
        "score_token_set_ratio": "Score from token set ratio", 
        "score_initials_partial_ratio": "Partial ratio score using initials", 
        "score_initials_used": "Is initials-based score used? (1=Yes,0=No)", 
        "website_with_highest_score": "Website with highest score for the given old ID (best website)", 
        "max_score": "Score of the website with highest score", 
        "is_max_score_initials_score": "Is the max score a initials-based score? (1=Yes,0=No)", 
        "is_max_score_avg_score": "Is the max score the average score? (1=Yes,0=No)",
        "assign_website": "Based on text similarity, is best website correctly assigned?(1=Yes,0=No)"
    }, 
    write_index = False
)

# 3. Fuzzy matching for Mexican new IDs with multiple old IDs and multimodal distribution in Aberdeen names for each new ID

## 3.1 Read Data

In [ ]:
# Read data of new IDs with multiple old IDs that have a multimodal distribution of aberdeen names for each new ID
mult_aberdeen_ind = pd.read_csv("../../Data/Mexico/processed_data/new_ids_cases_for_fuzzy_aberdeen_MEX.csv")
mult_aberdeen_ind[["new_id", "old_id", "domestic", "aberdeen_name"]].head(60)

## 3.2 Preprocessing

In [ ]:
# Remove common suffixes and other non-alphanumeric characters from name and website
mexican_suffixes = ["sa de cv", "s a de c v", "s de rl de cv", "sa cv", "s de rl", " sa ", "s. a. de c. v.",  
                "s.a.", "s.a. de c.v.", "inc", "corp", "llc", "r.l", "c.v", "s. de r.l.", " de ", " s.a. ", " s. ",
                    " de c.v.", "s.a. de c.v.", "sa de c v"]

def preprocess_aberdeen(name_panjiva, name_aberdeen):
    # Normalize: lowercase and remove accents
    name_panjiva = unidecode.unidecode(name_panjiva.lower())
    name_aberdeen = unidecode.unidecode(name_aberdeen.lower())
    
    # Remove common suffixes and other non-alphanumeric characters from name
    for suffix in mexican_suffixes:
        name_panjiva = name_panjiva.replace(suffix, '')
        name_aberdeen = name_aberdeen.replace(suffix, '')
    
    # Remove all non-alphanumeric characters from name
    name_panjiva = ''.join(e for e in name_panjiva if e.isalnum())
    name_aberdeen = ''.join(e for e in name_aberdeen if e.isalnum())


    return name_panjiva, name_aberdeen

## 3.3 Scoring

In [ ]:
# Empty dictionary to store results 
scores_aberdeen = {}
# Array with new IDs
new_ids = mult_aberdeen_ind["new_id"].unique()

# Iterate over new ids
for new_id in new_ids:
    # Filter dataset to get only observations of old ids associated with that new id 
    df_filtered = mult_aberdeen_ind[mult_aberdeen_ind["new_id"] == new_id]
    # Create an empty dictionary for the new id
    scores_aberdeen[new_id] = {}
    # Create an empty list to store websites with the highest scores
    best_names = []
    
    # Iterate over old ids-raw names associated with the new id
    for old_id, raw_name in zip(df_filtered.old_id.to_numpy(), df_filtered.domestic.to_numpy()):
        # Create an empty dictionary for each old id 
        scores_aberdeen[new_id][old_id] = {}
        # Assign raw name to the old id dictionary 
        scores_aberdeen[new_id][old_id][raw_name] = {}  
        
        max_score = 0
        best_name = None
        
        # Iterate over websites 
        for aberdeen_name in df_filtered.aberdeen_name.to_numpy():
            
            # Preprocess raw name and website and get initials of the cleaned name 
            clean_name, clean_aberdeen = preprocess_aberdeen(raw_name, aberdeen_name)
            
            # Scores comparing cleaned names and cleaned websites 
            score_ratio = fuzz.ratio(clean_name, clean_aberdeen)
            score_partial_ratio = fuzz.partial_ratio(clean_name, clean_aberdeen)
            score_token_sort_ratio = fuzz.token_sort_ratio(clean_name, clean_aberdeen)
            score_token_set_ratio = fuzz.token_set_ratio(clean_name, clean_aberdeen)
            
            # Average score 
            avg_score = (0.30*score_ratio + 0.40*score_partial_ratio + 0.15*score_token_sort_ratio + 0.15*score_token_set_ratio)
            
            
            if avg_score > max_score: 
                max_score = avg_score
                best_name = aberdeen_name
            
            scores_aberdeen[new_id][old_id][raw_name][aberdeen_name] = {
                "cleaned_name": clean_name,
                "cleaned_aberdeen": clean_aberdeen,
                "score_ratio": score_ratio,
                "score_partial_ratio": score_partial_ratio,
                "score_token_sort_ratio": score_token_sort_ratio,
                "score_token_set_ratio": score_token_set_ratio,
                "avg_score": avg_score, 
            } 
            
        
                
        # Store the best aberdeen name and its score for the current old_id
        scores_aberdeen[new_id][old_id]['best_name'] = {
            "aberdeen_name": best_name,
            "max_score": max_score, 
        }
        
        # Add the best name to the list 
        best_names.append(best_name)
        
        
    # Do all old IDs within the same new ID share the same "best" website?
    if len(set(best_names)) == 1:
        scores_aberdeen[new_id]["share_best_aberdeen_name"] = 1     
    else:
        scores_aberdeen[new_id]["share_best_aberdeen_name"] = 0  
      

In [ ]:
# Create an empty list to store data for DataFrame
data_list = []

# Iterate through the dictionary to extract required information
for new_id, old_ids in scores_aberdeen.items():
    for old_id, contents in old_ids.items():
        if old_id == "share_best_aberdeen_name":
            continue
        for panjiva_raw_name, aberdeen_names in contents.items():
            if panjiva_raw_name == "best_name":
                continue
            for aberdeen_name, info in aberdeen_names.items():
                data_list.append({
                    "new_id": new_id,
                    "old_id": old_id,
                    "panjiva_raw_name": panjiva_raw_name,
                    "cleaned_panjiva_name": info["cleaned_name"],
                    "aberdeen_name": aberdeen_name,
                    "cleaned_aberdeen_name": info["cleaned_aberdeen"],
                    "avg_score": info["avg_score"],
                    "score_partial_ratio": info["score_partial_ratio"],
                    "score_ratio": info["score_ratio"],
                    "score_token_sort_ratio": info["score_token_sort_ratio"],
                    "score_token_set_ratio": info["score_token_set_ratio"],
                    "aberdeen_name_with_highest_score": contents["best_name"]["aberdeen_name"],
                    "max_score": contents["best_name"]["max_score"],
                    "old_ids_share_best_aberdeen_name": old_ids["share_best_aberdeen_name"] 
                })

# Create a DataFrame from the list of dictionaries
df_aberdeen_multimodal = pd.DataFrame(data_list)
df_aberdeen_multimodal

In [ ]:
df_aberdeen_multimodal.head(60)

In [ ]:
# Share of new ids sharing the best aberdeen name
df_aberdeen_multimodal.drop_duplicates("new_id").old_ids_share_best_aberdeen_name.value_counts()/df_aberdeen_multimodal.drop_duplicates("new_id").old_ids_share_best_aberdeen_name.value_counts().sum()

In [ ]:
# Are all the max scores above the threshold? 
df_aberdeen_multimodal["all_max_scores_above_threshold"] = df_aberdeen_multimodal.groupby("new_id")["max_score"].transform(lambda x: (x > th_max_score).all()).astype(int)

# If the max score is above the threshold, then assign website to the new id
df_aberdeen_multimodal["assign_aberdeen_name"] = ((df_aberdeen_multimodal["all_max_scores_above_threshold"] == 1) & (df_aberdeen_multimodal["old_ids_share_best_aberdeen_name"] == 1)).astype(int)

In [ ]:
df_aberdeen_multimodal.assign_aberdeen_name.value_counts()

In [ ]:
df_aberdeen_multimodal[["new_id", "old_id", "panjiva_raw_name", "aberdeen_name", "cleaned_panjiva_name" ,"cleaned_aberdeen_name", "aberdeen_name_with_highest_score", "old_ids_share_best_aberdeen_name","assign_aberdeen_name"]].head(60)

In [ ]:
# Save data 
df_aberdeen_multimodal.to_stata( 
    path = "../../Data/Mexico/processed_data/fuzzy_aberdeen_names_scores_MEX.dta",
    variable_labels = {
        "new_id": "New ID",
        "old_id": "Old ID", 
        "panjiva_raw_name": "Panjiva raw name", 
        "cleaned_name": "Cleaned Panjiva raw name", 
        "aberdeen_name": "Aberdeen raw name", 
        "cleaned_aberdeen_name": "Cleaned Aberdeen name", 
        "avg_score": "Avg score(partial, partial ratio, token sort, set ratio)", 
        "score_partial_ratio": "Score from partial ratio", 
        "score_ratio": "Score from ratio", 
        "score_token_sort_ratio": "Score from token sort ratio", 
        "score_token_set_ratio": "Score from token set ratio", 
        "aberdeen_name_with_highest_score": "Aberdeen name with highest score for the given old ID (best aberdeen name)", 
        "max_score": "Score of the aberdeen name with highest score", 
        "old_ids_share_best_aberdeen_name": "Do old IDs share same top aberden name? (1=Yes,0=No)", 
        "all_max_scores_above_threshold":"For a given new ID, are all the max scores above the threshold (75)?", 
        "assign_aberdeen_name": "Based on text similarity, is best aberdeen name correctly assigned?(1=Yes,0=No)"
    }, 
    write_index = False
)

# 4. Fuzzy matching for new IDs with multiple old IDs but with only one old ID by new ID that was matched to Aberdeen (one Aberdeen name)

In this section, we analyze cases where there are new IDs with multiple old IDs, but it was only possible to retrive the website for one of these old IDs, i.e., only one old ID was matched to Builtwith. 

In [ ]:
# Read data of new IDs with multiple old IDs that have only one aberdeen name matched. This is different from the 
# unimodal case, as these are new IDs that have the flag of multiple old IDs -- because in the correspondence table they 
# do have multiple aberdeen names -- but only the panjiva name of one of these old IDs was able to be matched to Aberdeen  
only_one_aberdeen_name = pd.read_csv("../../Data/Mexico/processed_data/one_aberdeen_name_matched_cases_for_fuzzy_MEX.csv").drop("Unnamed: 0", axis = 1)[["old_id", "new_id", "domestic", "aberdeen_name", "only_one_aberdeen_matched"]]

## 4.2. Preprocessing 

We follow similar preprocessing steps as in the section 1.2. to clean Panjiva raw names and Aberdeen names. 

In [ ]:
only_one_aberdeen_name[["cleaned_name", "cleaned_aberdeen_name"]] = only_one_aberdeen_name.apply(lambda x: preprocess_aberdeen(x["domestic"], x["aberdeen_name"]), axis=1, result_type = "expand")
only_one_aberdeen_name

## 4.3 Scoring Algorithm 

In [ ]:
# Function to compute Scores
def compute_scores_aberdeen(clean_name, clean_aberdeen):
    
    # Scores comparing cleaned names and cleaned websites 
    score_ratio = fuzz.ratio(clean_name, clean_aberdeen)
    score_partial_ratio = fuzz.partial_ratio(clean_name, clean_aberdeen)
    score_token_sort_ratio = fuzz.token_sort_ratio(clean_name, clean_aberdeen)
    score_token_set_ratio = fuzz.token_set_ratio(clean_name, clean_aberdeen)
            
    # Average score 
    avg_score = (score_ratio + score_partial_ratio + score_token_sort_ratio + score_token_set_ratio) / 4
                        
    if (score_partial_ratio > avg_score) and (score_partial_ratio >= th_partial):  # Create a threshold to know when to use the score_partial_ratio
        max_score = score_partial_ratio
    else: 
        max_score = avg_score

    return score_ratio, score_partial_ratio, score_token_sort_ratio, score_token_set_ratio, avg_score, max_score 

In [ ]:
# Create variables with scores 
only_one_aberdeen_name[
    ["score_ratio", "score_partial_ratio",
     "score_token_sort_ratio","score_token_set_ratio", 
     "avg_score", "max_score"]
] = only_one_aberdeen_name.apply(lambda x: compute_scores_aberdeen(x["cleaned_name"], x["cleaned_aberdeen_name"]), 
                       axis =1, result_type = "expand")


# Threshold to assign website to panjiva raw name:  
only_one_aberdeen_name[["assign_aberdeen_name"]] = (only_one_aberdeen_name[["max_score"]] > th_max_score).astype(int)

In [ ]:
only_one_aberdeen_name.assign_aberdeen_name.value_counts()

In [ ]:
only_one_aberdeen_name.columns

In [ ]:
only_one_aberdeen_name[["domestic", "aberdeen_name", "assign_aberdeen_name"]].head(60)

In [ ]:
# Save data
only_one_aberdeen_name.to_stata( 
    path = "../../Data/Mexico/processed_data/fuzzy_one_aberdeen_name_scores_MEX.dta",
    variable_labels = {
        "new_id": "New ID",
        "old_id": "Old ID", 
        "panjiva_raw_name": "Panjiva raw name", 
        "cleaned_name": "Cleaned Panjiva raw name", 
        "aberdeen_name": "Builtwith website", 
        "cleaned_aberdeen_name": "Cleaned Builtwith website", 
        "avg_score": "Avg score(partial, partial ratio, token sort, set ratio)", 
        "score_partial_ratio": "Score from partial ratio", 
        "score_ratio": "Score from ratio", 
        "score_token_sort_ratio": "Score from token sort ratio", 
        "score_token_set_ratio": "Score from token set ratio", 
        "score_initials_partial_ratio": "Partial ratio score using initials", 
        "score_initials_used": "Is initials-based score used? (1=Yes,0=No)", 
        "max_score": "Score of the website with highest score", 
        "is_max_score_initials_score": "Is the max score a initials-based score? (1=Yes,0=No)", 
        "assign_aberdeen_name": "Based on text similarity, is the aberdeen name correctly assigned?(1=Yes,0=No)"
    }, 
    write_index = False
)